# Writing files to minio S3

In [1]:
import fsspec
from asammdf import MDF

### ```minio``` configs.

The many ways to use a custom S3 endpoint url:

In [2]:
s3_cfg = {
    "key": "minioadmin",
    "secret": "minioadmin",
    "client_kwargs": {
        "endpoint_url": "http://127.0.0.1:9000",
    },
}
fs = fsspec.filesystem("s3", **s3_cfg)

ModuleNotFoundError: No module named 's3fs'

In [3]:
fs.ls("AerospaceStartup")

NameError: name 'fs' is not defined

In [ ]:
with fs.open("AerospaceStartup/hello.txt", "wt") as fid:
    fid.write("Hello World")

In [ ]:
fs.ls("AerospaceStartup")

# Walk Through All Files:

Walk through all S3 files and find the first one.

In [ ]:
?timeit

In [ ]:
%%timeit
import os
mdf_paths=list()
for company in fs.ls(""):
    for root, dirs, files in fs.walk(company):
        for file in files:
            if file.lower().endswith(".mf4") or file.lower().endswith(".mdf"):
                mdf_paths.append(os.path.join(root, file))

In [ ]:
len(mdf_paths)

Loop through all mdf files found:

In [ ]:
for mdf_path in mdf_paths:
    break

Print off the information ```fsspec.info```  contains:

In [ ]:
fs.info(mdf_path)

Goal 1: Find the largest MDF file indexed through minio.

Get all the MDF file sizes:

In [ ]:
sizes = [fs.info(mdf_path)["size"] for mdf_path in mdf_paths]

In [ ]:
# https://stackoverflow.com/a/6423325
myList = [1, 2, 3, 100, 5]    
sorted(range(len(myList)),key=myList.__getitem__)

In [ ]:
biggest_idx = sorted(range(len(sizes)),key=sizes.__getitem__)[-1]
biggest_idx

In [ ]:
mdf_paths[biggest_idx]

In [ ]:
fs.info(mdf_paths[biggest_idx])

In [ ]:
biggest_mdf = fs.info(mdf_paths[biggest_idx])

GB:

In [ ]:
biggest_mdf["size"]/1024**3

MB:

In [ ]:
biggest_mdf["size"]/1024**2

In [ ]:
# https://github.com/dask/s3fs/issues/273
# https://github.com/pandas-dev/pandas/pull/29050
%env AWS_ACCESS_KEY_ID="minioadmin"
%env AWS_SECRET_ACCESS_KEY="minioadmin"
%env S3_ENDPOINT="http://127.0.0.1:9000"

fs2=s3fs.S3FileSystem(
    client_kwargs={
        "endpoint_url": "http://127.0.0.1:9000",
    })
fs2.ls("")

In [ ]:
import asammdf

In [ ]:
%%timeit
of = fs.open(mdf_paths[biggest_idx])
mdf = asammdf.MDF(of)

In [ ]:
of = fs.open(mdf_paths[biggest_idx])

mdf = asammdf.MDF(of)

In [ ]:
mdf

In [ ]:
mdf.channels_db

In [ ]:
x = mdf.get_group(0)

# Convert to fsspec filecache

Exact example from anaconda article: [Introducing Remote Content Caching with FSSpec.](https://www.anaconda.com/fsspec-remote-caching/)

In [ ]:
import fsspec
of = fsspec.open("filecache://anaconda-public-datasets/iris/iris.csv", mode='rt', 
                 cache_storage='/tmp/cache1',
                 target_protocol='s3', target_options={'anon': True})
with of as f:
    print(f.readline())

In [ ]:
cache_dir = "/tmp/mdf_cache"
fsspec_kwargs = {
    "urlpath": f"filecache://{mdf_path}",
    "mode": 'rb', 
    "cache_storage": cache_dir,
    "target_protocol": 's3',
    "target_options": s3_cfg,
}

In [ ]:
import shutil
import time

In [ ]:
shutil.rmtree(
    path=cache_dir,
    ignore_errors=True
)
t1=time.time()
with fsspec.open(**fsspec_kwargs) as of:
    mdf = MDF(of)
t2=time.time()
with fsspec.open(**fsspec_kwargs) as of:
    mdf2 = MDF(of)
t3=time.time()

print(f"Uncached Read: {t2-t1}s")
print(f"Cached Read: {t3-t2}s")

# All S3 Files Cached/Uncached.

In [ ]:
import os
mdf_paths=list()
for root, dirs, files in fs.walk("canedge-live-demo-2"):
    for file in files:
        if file.lower().endswith(".mf4"):
            mdf_paths.append(os.path.join(root, file))
len(mdf_paths)

In [ ]:
mdfs_uncached = list()
mdfs_cached = list()

cache_dir = "/tmp/mdf_cache"

shutil.rmtree(
    path=cache_dir,
    ignore_errors=True
)
t1=time.time()
for mdf_path in mdf_paths:
    fsspec_kwargs = {
        "urlpath": f"filecache://{mdf_path}",
        "mode": 'rb', 
        "cache_storage": cache_dir,
        "target_protocol": 's3',
        "target_options": s3_cfg,
    }
    with fsspec.open(**fsspec_kwargs) as of:
        mdfs_uncached.append(MDF(of))  
        
t2=time.time()

for mdf_path in mdf_paths:
    fsspec_kwargs = {
        "urlpath": f"filecache://{mdf_path}",
        "mode": 'rb', 
        "cache_storage": cache_dir,
        "target_protocol": 's3',
        "target_options": s3_cfg,
    }
    with fsspec.open(**fsspec_kwargs) as of:
        mdfs_cached.append(MDF(of))  
t3=time.time()

print(f"Uncached Read: {t2-t1}s")
print(f"Cached Read: {t3-t2}s")